# Inverse Kinematics

### Developed as a part of the Dartmouth CS70 curriculum
### Author: Jacob Bacus
---

This notebook contains 4 functions along with relevant import statements and demos. 
The functions are as follows:

- `chain` represents a collection of segments using forward kinematics to solve for positions. 

- `dchain` finds the jacobian of for a chain (taking in an array of angles)

- `newton` Uses Newton's Method to solve the inverse kinematics problem of the chain updating of segments in the chain based on the error from the endpoint (uses pseudoinverse for calculation)

- `ik_demo` provides a live demo of a chain being moved based on inverse kinematics. Two examples are given, a chain with 4 segments and a chain with 30. The red endpoint can be dragged and the chain will adjust accordingly. Note that at high segment counts, fast movements will become difficult to calculate and the chain may fail to reach the desired endpoint.

In [21]:
import numpy as np
import scipy.linalg as la

# New: Optimization package
import scipy.optimize as opt

# bqplot plotting library
import bqplot as bqp

# Import graphical user interface components used below
from ipywidgets import interact
from ipywidgets import FloatSlider, VBox

In [4]:
def chain(theta):
    if len(theta) == 0:
        return np.array([0., 0.])
    else:
        # Goes to the last joint until it hits the end
        prev_pos = chain(theta[:-1])
        # Compute the position of the current joint
        x = prev_pos[0] + np.cos(np.sum(theta))
        y = prev_pos[1] + np.sin(np.sum(theta))
        return np.array([x, y])

# TESTS
print('chain:     ', chain([0.1, 0.2, 0.3, 0.4]))
print('reference: ', np.array([ 3.31597858,  1.80146708]))
print()
print('chain:     ', chain([np.pi, np.pi, np.pi, np.pi]))
print('reference: ', np.array([0, 0]))

chain:      [3.31597858 1.80146708]
reference:  [3.31597858 1.80146708]

chain:      [ 0.0000000e+00 -2.4492936e-16]
reference:  [0 0]


In [5]:
def dchain(theta):
    n = len(theta)
    J = np.zeros((2, n))  # Initial matrix
    
    # 
    angle_sum = np.sum(theta)  # Start with sum of angles
    x = 0
    y = 0
    
    # Loop over each joint from the last to the first
    for i in range(n-1, -1, -1):
        # Update the x and y position
        x += np.cos(angle_sum)
        y += np.sin(angle_sum)
        
        # Calculate the partial derivatives
        J[0, i] = -y
        J[1, i] = x   
        
        # Subtract the from total angle
        angle_sum -= theta[i]
    
    return J

# TESTS
print('dchain: \n', dchain([0, 0, 0, 0]))
print('reference: \n', np.array([[ 0.,  0.,  0.,  0.], [ 4.,  3.,  2.,  1.]]))
print()
print('dchain: \n', dchain([0.1, 0.2, 0.3]))
print('reference: \n', np.array([[-0.9599961 , -0.86016268, -0.56464247], [ 2.77567627,  1.7806721 ,  0.82533561]]))

dchain: 
 [[-0. -0. -0. -0.]
 [ 4.  3.  2.  1.]]
reference: 
 [[0. 0. 0. 0.]
 [4. 3. 2. 1.]]

dchain: 
 [[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]
reference: 
 [[-0.9599961  -0.86016268 -0.56464247]
 [ 2.77567627  1.7806721   0.82533561]]


In [6]:
def newton(t, target):
    for i in range(8):  # Perform 8 iterations
        # Compute the current position of guess
        current_position = chain(t)
        
        # Calculate difference in our position and target
        error = current_position - target
        
        # Compute the Jacobian matrix for current angles
        J = dchain(t)
        
        # Use psuedoinverse to update our current guess
        t -= la.pinv(J) @ error
    
    return t

# TESTS
# Moving a 1-element chain from the default configuration to position (0, 1)
print('chain(newton): ', chain(newton(np.array([0.]), np.array([0., 1.]))))
print('reference:     ', np.array([0, 1]))
print()
# Moving a 2-element chain from the default configuration to position (0.5, 0.5)
print('chain(newton): ', chain(newton(np.array([0., 0.]), np.array([0.5, 0.5]))))
print('reference:     ', np.array([0.5, 0.5]))

chain(newton):  [6.123234e-17 1.000000e+00]
reference:      [0 1]

chain(newton):  [0.5 0.5]
reference:      [0.5 0.5]


In [7]:
def ik_demo(solver, size):    
    theta = np.zeros(size, dtype=np.float64)
    
    # Function which repeatedly calls ``chain`` to compute all joint positions
    def chain_all(theta):
        return np.column_stack([chain(theta[:i]) for i in range(0, len(theta) + 1)])

    # Callback that is invoked when the user drags the red endpoint around
    def refresh(_):
        # 'theta' is a variable of the parent function, we want to modify it here
        nonlocal theta
        
        # Target position
        target = np.array([scat2.x[0], scat2.y[0]])
        
        # Don't try to solve the problem if the user dragged the point out of the circle
        if la.norm(target) > size:
            return
        
        # Call the provided IK solver
        theta = solver(theta, target)
        
        # Update the positions
        values = chain_all(theta)
        scat.x, scat.y = values
        lines.x, lines.y = values
    
    # Similar to fk_solver(), create a number of plots and merge them
    scales = { 'x': bqp.LinearScale(min=-size-1, max=size+1),
               'y': bqp.LinearScale(min=-size-1, max=size+1) }

    scat  = bqp.Scatter(scales=scales)
    lines = bqp.Lines(scales=scales)

    # Create a circle which marks the boundary of where the red point can be moved
    circle_x = np.cos(np.linspace(0, 2*np.pi, 100)) * size
    circle_y = np.sin(np.linspace(0, 2*np.pi, 100)) * size
    circle = bqp.Lines(x=circle_x, y=circle_y,
                       scales=scales, colors=['gray'])
    
    # Special plot, which contains the red endpoint that can be moved
    scat2 = bqp.Scatter(scales=scales,
                        enable_move=True, 
                        update_on_move=True,
                        colors=['red'])

    # Initialize the visualizations with the default configuration
    values = chain_all(theta)
    scat.x, scat.y = values
    lines.x, lines.y = values
    scat2.x, scat2.y = chain(theta).reshape(2, 1)
    
    # Call the 'refresh' function when the red dot is moved
    scat2.observe(refresh, names=['x', 'y'])

    figure = bqp.Figure(marks=[scat, scat2, lines, circle])
    figure.layout.height = '500px'
    figure.layout.width = '500px'
    return figure

In [8]:
ik_demo(newton, 4)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='500…

In [10]:
ik_demo(newton, 30)

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(height='500px', width='500…